In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
THREADS=3
tf.config.threading.set_intra_op_parallelism_threads(THREADS)
tf.config.threading.set_inter_op_parallelism_threads(THREADS)
import random
import numpy as np
from numpy import genfromtxt
import pandas as pd
import base64
import sys
import glob, os
import re
import subprocess
import datetime
import queue
from IPython.display import clear_output
from tensorflow.python.client import device_lib

print("Python:"+sys.version)
print("TF:"+tf.__version__)
print("GPU:"+str(tf.test.is_gpu_available())+" CUDA:"+str(tf.test.is_built_with_cuda()))
#print(device_lib.list_local_devices())
import os
if not os.path.exists('/run/shm/traindata'):
    os.makedirs('/run/shm/traindata')


In [ ]:
MATCHES_PER_GENERATION=800
PIT_MATCHES=400
THREADS=4
#G7_TRAINING_POOL_SIZE=800000
#TRAINING_POOL_SIZE=600000
TRAINING_POOL_SIZE=3500000  #gen 157 suffering low diversity
K_BATCH_SIZE=32
K_ITERATIONS=650
K_EPOCH=20
#G7_TRAINING_SUBSET_SIZE=600000
TRAINING_SUBSET_SIZE=350000
K_WEIGHT_POLICY=1.0
K_WEIGHT_VALUE=1.0
TRAINING_LOOPS=50
WINRATE_ACCEPTED=55.6
INPUT_SIZE=6*2*24+2*27
POLICY_SIZE=6

LEARNING_RATE_MIN=0.00005
LEARNING_RATE_MAX=0.00021
#K_LEARNING_RATE=0.00007
#G7_K_LEARNING_RATE=0.005
K_LEARNING_RATE=0.001
#up to gen9 K_LEARNING_RATE=0.0007 
#K_LEARNING_RATE=0.00009 #gen 10 

PROPAGATE_BASE=0.90 #f; //Propagate "WIN/LOSS" with 10% at start
PROPAGATE_INC=0.10#1.0-PROPAGATE_BASE #f; //Linearly increase  until endgame
#PROPAGATE_BASE=0.20 #f; //Propagate "WIN/LOSS" with 10% at start
#PROPAGATE_INC=1.0-PROPAGATE_BASE #f; //Linearly increase  until endgame
POLICY_BACKP_FIRST=20 #; //Similarly , but with percentage of turns, first 30% of turns doesn't have any "temperature",
POLICY_BACKP_LAST=5 #; //from 30% to (100-10=90%) I linearly sharpen policy to get only the best move, 

K_L2=0.00002
TRAIN_CPUCT_MIN=3.00
TRAIN_CPUCT_INC=0.00
TRAIN_CPUCT_MAX=3.00
#gen136
TRAIN_CPUCT_MIN=2.8965516739437245
TRAIN_CPUCT_INC=0.0034086042277349836 
TRAIN_CPUCT_MAX=2.974563702328649

TRAIN_NOISE_DIR_EPSILON=0.20
#TRAIN_NOISE_DIR_ALPHA=1.3
#TRAIN_NOISE_DIR_DECAY=0.0007
#TRAIN_NOISE_DIR_DECAY=0.0002 #gen 136
TRAIN_NOISE_DIR_ALPHA=1.6 #gen 154
TRAIN_NOISE_DIR_DECAY=0.0 #gen 154
TRAIN_NOISE_DIR_EPSILON=0.24 #gen 174
TRAIN_NOISE_DIR_ALPHA=1.5 #gen 174
TRAIN_NOISE_DIR_DECAY=0.0 #gen 174

TRAIN_MCTS_ITER=2000
TRAIN_NOISE_RANDOM=0.0
TRAIN_NOISE_RANDOM=0.05 #gen 174
TRAIN_PARAMS = f"{TRAIN_CPUCT_MIN} {TRAIN_CPUCT_INC} {TRAIN_CPUCT_MAX} {TRAIN_MCTS_ITER} {TRAIN_NOISE_DIR_EPSILON} {TRAIN_NOISE_DIR_ALPHA} {TRAIN_NOISE_DIR_DECAY} {TRAIN_NOISE_RANDOM} {PROPAGATE_BASE} {PROPAGATE_INC} {POLICY_BACKP_FIRST} {POLICY_BACKP_LAST}"

PIT_CPUCT_MIN=3.00
PIT_CPUCT_INC=0.00
PIT_CPUCT_MAX=3.00
#gen136
PIT_CPUCT_MIN=2.8965516739437245
PIT_CPUCT_INC=0.0034086042277349836 
PIT_CPUCT_MAX=2.974563702328649

#PIT_NOISE_DIR_EPSILON=0.04 
PIT_NOISE_DIR_EPSILON=0.04  #gen 174
PIT_NOISE_DIR_ALPHA=1.3
PIT_NOISE_DIR_DECAY=0.0007
PIT_MCTS_ITER=3000
PIT_NOISE_RANDOM=0.02  #gen 174
PIT_PARAM_THREAD= f"{THREADS} {PIT_MATCHES}"
PIT_PARAM_MCTS = f"{PIT_CPUCT_MIN} {PIT_CPUCT_INC} {PIT_CPUCT_MAX} {PIT_MCTS_ITER} {PIT_NOISE_DIR_EPSILON} {PIT_NOISE_DIR_ALPHA} {PIT_NOISE_DIR_DECAY} {PIT_NOISE_RANDOM} {PROPAGATE_BASE} {PROPAGATE_INC} {POLICY_BACKP_FIRST} {POLICY_BACKP_LAST}"
SAMPLES_FILE=os.path.join(".","traindata","samples.dat")
sampler_process=os.path.join(".","NNSampler")+" "+os.path.join(".","traindata")+" Replay.*.dat "+SAMPLES_FILE+" "+str(TRAINING_POOL_SIZE)+" "+str(TRAINING_SUBSET_SIZE)+" "+str(INPUT_SIZE)+" "+str(1+POLICY_SIZE)+" 1"
print(LEARNING_RATE_MAX)


In [ ]:
def adjustLR():
    if generation == 0:
        return 0.008
    elif generation < 5:
        return 0.006
    elif generation < 10:
        return 0.002
    else:
        return 0.0006
def adjustLR2():
    if generation == 0:
        return 0.005
    elif generation < 5:
        return 0.001
    elif generation < 10:
        return 0.0007 
    else:
        return 0.00008

def policy_loss(y_true, y_pred): 
    p_true = tf.where(y_true >= 0.0, y_true, 0)
    p_pred = tf.where(y_true >= 0.0, y_pred, -99999999.99)
    p_pred = tf.nn.softmax(p_pred)
    #this is wrong, categorical_crossentropy should be tf.keras.losses.KLD
    return 0.8*tf.keras.losses.categorical_crossentropy(p_true, p_pred)+0.2*tf.keras.losses.categorical_crossentropy(y_true, y_pred)
def learning_rate_scheduler(epoch, lr):
    if (generation<4):
        return 0.02-0.003*generation
    else:
        LR_RANGE=LEARNING_RATE_MAX-LEARNING_RATE_MIN
        MID_EPOCH=K_ITERATIONS*0.5
        LR_STEP=LR_RANGE/MID_EPOCH        
        DIST_EPOCH=abs(MID_EPOCH-epoch)
        L = LEARNING_RATE_MAX -LR_STEP*DIST_EPOCH
        if (generation<15):
            return max(L,LEARNING_RATE_MIN)*2.0
        else:
            return min(max(L,LEARNING_RATE_MIN),LEARNING_RATE_MAX)
def SaveModel(my_model,fileSTR):
    totalbytes=0
    data=[]
    Wmodel = open("./"+fileSTR, "wb")
    for x in my_model.weights:
        nn = x.numpy()
        T = nn
        v = np.ndarray.tobytes(T)
        Wmodel.write(bytearray(v))
        totalbytes+=len(v)
        data.append(base64.b64encode(v).decode("utf-8"))
    Wmodel.close()
def readWinrate(candidatefile,bestfile):
    files=sorted(glob.glob(os.path.join('pitresults','Pit_'+candidatefile+'_'+bestfile+'_*.txt')),reverse=True)
    for file in files:
        with open(file, 'r') as f:
            strvalue=f.read().strip()
        return float(strvalue)
    return -1.0
def readAndDeleteWinrate(candidatefile,bestfile):
    files=sorted(glob.glob(os.path.join('pitresults','Pit_'+candidatefile+'_'+bestfile+'_*.txt')),reverse=True)
    valor=-1.0
    for file in files:
        if (valor == -1.0):
            with open(file, 'r') as f:
                valor=float(f.read().strip())
        os.remove(file)
    return valor

LR_callback = tf.keras.callbacks.LearningRateScheduler(learning_rate_scheduler)
LR_decay = tf.keras.callbacks.LearningRateScheduler(tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-3,decay_steps=20000,decay_rate=0.9))

In [ ]:
inputs =  tf.keras.Input(shape=(INPUT_SIZE,), name='input')
x = tf.keras.layers.Dense(TODOTODOTODO,activation='relu',name='Dense1')(inputs)
#x = tf.keras.layers.Dense(TODOTODOTODO,activation='relu')(x)

p1 = tf.keras.layers.Dense(TODOTODOTODO,activation='relu',name='p1')(x)
#p1 = tf.keras.layers.Dense(TODOTODOTODO,activation='relu')(p1)
v1 = tf.keras.layers.Dense(TODOTODOTODO,activation='relu',name='v1')(x)
#v1 = tf.keras.layers.Dense(TODOTODOTODO,activation='relu')(v1)
value = tf.keras.layers.Dense(1, activation='tanh',name='value')(v1)
policy = tf.keras.layers.Dense(POLICY_SIZE, activation='softmax',name='policy')(p1)
model = tf.keras.Model(inputs=inputs, outputs=[value, policy])

#opt = tf.keras.optimizers.Adam(learning_rate=K_LEARNING_RATE)
opt = tf.keras.optimizers.SGD(learning_rate=K_LEARNING_RATE)
csv_logger = tf.keras.callbacks.CSVLogger('gen_train.log',append=True)

model.compile(loss={'value': 'mean_squared_error',
                    #'policy': policy_loss
                    #'policy':'categorical_crossentropy'
                    'policy':tf.keras.losses.KLD
                   },
              optimizer=opt,
              loss_weights = {'value':K_WEIGHT_VALUE,
                              'policy':K_WEIGHT_POLICY}
              #,steps_per_execution=TRAINING_LOOPS,
              ,metrics={'value':'mean_absolute_percentage_error',
                        #'value':'mean_absolute_error',
                       'policy': 'categorical_accuracy' }
             )

In [ ]:
if not os.path.exists('gen0000.h5'):
    model.save('gen0000.h5')
    SaveModel(model,"gen0000.w32")
    model.summary()

In [ ]:
generation=0
gen_name="gen"+str(generation).zfill(4)
if os.path.exists('generation.txt'):
    with open('generation.txt', 'r') as f:
        generation = int(f.read().strip())
        gen_name="gen"+str(generation).zfill(4)
        print("Generation is :"+gen_name+" "+str(generation))        
        if (generation > 0):
            model =  tf.keras.models.load_model(gen_name+'.h5', custom_objects={"policy_loss": policy_loss})
if not os.path.exists(gen_name+".w32"):
    SaveModel(model,gen_name+".w32")
    model.save(gen_name+'.h5')
gen_best1=gen_name
if os.path.exists('gen_best1.txt'):
    with open('gen_best1.txt', 'r') as f:
        gen_best1 = f.read().strip()
gen_best2=gen_name
if os.path.exists('gen_best2.txt'):
    with open('gen_best2.txt', 'r') as f:
        gen_best2 = f.read().strip()        
model=tf.keras.models.load_model(gen_name+'.h5', custom_objects={"policy_loss": policy_loss})
print("Best Model1:"+gen_best1+" + "+gen_best2+". Current generation:"+gen_name+" loaded")


In [ ]:
clear_output(wait=True)
try:
    pit_winrate2
except NameError:
    pit_winrate2=70.0
try:
    pit_winrate
except NameError:
    pit_winrate=max(0,100.0-pit_winrate2)

pit_winrate2=70
pit_winrate=70
    
while True:
    model.optimizer.learning_rate.assign(adjustLR())
    if (generation % 10 == 0):
        clear_output(wait=True)
    samplescount=0
    #if (generation == 256):
     #   csv_data = np.fromfile(SAMPLES_FILE, dtype=np.float32)
      #  csv_data=np.reshape(csv_data, (-1,INPUT_SIZE+POLICY_SIZE+2))
       # samplescount =(csv_data.shape)[0]
    #generate enough diversity to train
    while samplescount < TRAINING_SUBSET_SIZE/3:
        if (generation==0):
            random_enemy=gen_name
        else:
            random_enemy="gen"+str(random.randint(max(1,generation-5), generation)).zfill(4)
        randpick=random.randint(0,100)

        if (generation==0 and (len(glob.glob(os.path.join('traindata','Replay_'+'*'+gen_name+"vs"+gen_name+'.dat')))>0)):
            print('Replay_'+'*'+gen_name+'.txt already exists') 
        else:
            pFirst=max(0.2,0.9*(1.0-(pit_winrate/(pit_winrate+pit_winrate2))))
            pSecond=0.9-pFirst
            print(f" **** Doing samples. Count:{samplescount}. pBest1:{100.0*pFirst}% p2:{100.0*pSecond}%  {pit_winrate} {pit_winrate2}")
            p70=int(pFirst*MATCHES_PER_GENERATION)
            p20=int(pSecond*MATCHES_PER_GENERATION)
            p5=MATCHES_PER_GENERATION-p70-p20
            selfplay_process=os.path.join(".","CGZero")+" selfplay "+f"{THREADS} {p70} "+gen_best1+" "+TRAIN_PARAMS+" "+gen_best1+" "+TRAIN_PARAMS
            print(selfplay_process)
            subprocess.run(selfplay_process, shell=True)
            selfplay_process=os.path.join(".","CGZero")+" selfplay "+f"{THREADS} {p20} "+gen_best1+" "+TRAIN_PARAMS+" "+gen_best2+" "+TRAIN_PARAMS
            print(selfplay_process)
            subprocess.run(selfplay_process, shell=True)
            if (p70 >= p20):
                A=gen_best1 if (gen_best1 >= random_enemy) else random_enemy
                B=gen_best1 if (A == random_enemy) else random_enemy
            else:
                A=gen_best2 if (gen_best2 >= random_enemy) else random_enemy
                B=gen_best2 if (A == random_enemy) else random_enemy
            selfplay_process=os.path.join(".","CGZero")+" selfplay "+f"{THREADS} {p5} "+A+" "+TRAIN_PARAMS+" "+B+" "+TRAIN_PARAMS
            print(selfplay_process)
            subprocess.run(selfplay_process, shell=True)
        print('Reading training data')
        print(sampler_process)
        subprocess.run(sampler_process, shell=True)
        csv_data = np.fromfile(SAMPLES_FILE, dtype=np.float32)
        csv_data=np.reshape(csv_data, (-1,INPUT_SIZE+POLICY_SIZE+2))
        samplescount =(csv_data.shape)[0]
    np.random.shuffle(csv_data)
    cut_index = [(csv_data.shape)[1]-POLICY_SIZE-2, (csv_data.shape)[1]-2,(csv_data.shape)[1]-1]
    samples,policy,value,countVisits=np.split(csv_data, cut_index,axis=1)
    
    mask= np.where(policy < 0, -999999999.99, policy)
    policy= np.where(policy < 0, 0, policy) 
  
    #1000 minibatches
    #for loop in range(K_ITERATIONS):
     #   print("Batch "+str(loop)+":",end='')
      #  indices = np.random.choice(value.shape[0], K_BATCH_SIZE, replace=False)
       # S=samples[indices]
       # P=policy[indices]
       # V=value[indices]
    #    #model.optimizer.learning_rate.assign(learning_rate_scheduler(loop, 0.0))
    #    #model.optimizer.learning_rate.assign(K_LEARNING_RATE)
       # model.fit({'input':S}, {'policy': P, 'value':V},verbose=2, epochs=K_EPOCH,callbacks=[csv_logger],batch_size=int(K_BATCH_SIZE/4))
    model.fit({'input':samples}, {'policy': policy, 'value':value},verbose=2, epochs=K_EPOCH,callbacks=[csv_logger],batch_size=int(K_BATCH_SIZE/4))    
    print('New generation '+gen_name+' -> '+"gen"+str(generation+1).zfill(4))
    #new generation
    generation=generation+1
    gen_name="gen"+str(generation).zfill(4)
    #Saving
    print('Save Model '+gen_name+'.h5')
    model.save(gen_name+'.h5')
    SaveModel(model,gen_name+".w32")
    with open('generation.txt', 'w') as f:
        f.write(str(generation))    
    #pit
    pitplay_process=os.path.join(".","CGZero")+" pitplay "+PIT_PARAM_THREAD+" "+gen_name+" "+PIT_PARAM_MCTS+" "+gen_best1+" "+PIT_PARAM_MCTS
    print('subprocess.run('+pitplay_process+', shell=True)')
    subprocess.run(pitplay_process, shell=True)
    #clear_output(wait=True)
    #read pit value
    pit_winrate=readWinrate(gen_name,gen_best1)
    pitplay_process=os.path.join(".","CGZero")+" pitplay "+PIT_PARAM_THREAD+" "+gen_name+" "+PIT_PARAM_MCTS+" "+gen_best2+" "+PIT_PARAM_MCTS
    subprocess.run(pitplay_process, shell=True)
    pit_winrate2=readWinrate(gen_name,gen_best2)
    print('Winrate '+str(pit_winrate)+' '+str(pit_winrate2))
    if (pit_winrate>=WINRATE_ACCEPTED):
        print("New best:"+gen_name+" vs "+gen_best1+": Winrate:"+str(pit_winrate)+"%")
        #now vs best2
        if gen_best1 != gen_best2:
            print("        :"+gen_name+" vs "+gen_best2+": Winrate:"+str(pit_winrate2)+"%")
            tmpgenbest1=gen_best1
            gen_best1=gen_name
            with open('gen_best1.txt', 'w') as f:
                f.write(gen_best1)
        else:
            pit_winrate2 = 49.0
        if (pit_winrate2>=50.0):
            gen_best2=tmpgenbest1
            with open('gen_best2.txt', 'w') as f:
                f.write(gen_best2)
    #if (pit_winrate<=100.0-(WINRATE_ACCEPTED)):
     #   print("WORST:"+gen_name+" vs "+gen_best1+": Winrate:"+str(pit_winrate)+"% Recover Best generation "+gen_best1)
      #  model=tf.keras.models.load_model(gen_best1+'.h5')


In [ ]:
def softmax(xs):
    return np.exp(xs) / sum(np.exp(xs))

xs = np.array([0.110082 ,0.129164 ,0.147099 ,0.136864 ,0.0711029 ,0.405688])
print(softmax(xs))

In [ ]:

#SaveModel(model,"prueba.w32")
#predictions = model.predict(samples)
print(samples[8])
print("Value Predicted:",end='')
print(predictions[0][8])

print("Policy Predicted:",end='')
print(predictions[1][8])

#mogen=tf.keras.models.load_model('gen0000.h5', custom_objects={"policy_loss": policy_loss})
print(samples)

In [ ]:
def lr2(epoch, lr):
    return 0.02
    #return 0.001
    #return 0.001
print(samples.shape)
#print(mogen.optimizer.lr.read_value())
mogen=model #tf.keras.models.load_model('gen0000.h5', custom_objects={"policy_loss": policy_loss})
#mogen.optimizer.lr.assign(0.02)
mogen.fit({'input':samples}, {'policy': policy, 'value': value},verbose=2, epochs=150,batch_size=64)
#mogen.optimizer.lr.assign(0.005)
mogen.fit({'input':samples}, {'policy': policy, 'value': value},verbose=2, epochs=150,batch_size=64)
#mogen.optimizer.lr.assign(0.001)
mogen.fit({'input':samples}, {'policy': policy, 'value': value},verbose=2, epochs=150,batch_size=64)
#Adam loss: 0.8129 - value_loss: 0.1733 - policy_loss: 0.6396 

In [ ]:
predictions = mogen.predict(inputs)
print(len(policy))
for n in range(30):
    nv = n
    print(str(nv)+" Value Predicted:"+str(abs(predictions[0][nv][0]-value[nv][0]))+"  | "+str(predictions[0][nv][0])+' '+str(value[nv][0]))
    print(str(nv)+" Policy Predicted:",end='')
    np.set_printoptions(suppress=True)
    np.set_printoptions(precision=1)
    print(predictions[1][nv],end='')
    np.set_printoptions(precision=1)
    print(policy[nv])
    np.set_printoptions(precision=3)

In [ ]:
csv_data = np.fromfile('samples.dat', dtype=np.float32)

In [ ]:
csv_data = np.fromfile(SAMPLES_FILE, dtype=np.float32)

csv_data = np.fromfile('./traindata3/Replay_gen0256_gen0256.dat', dtype=np.float32)
csv_data=np.reshape(csv_data, (-1,INPUT_SIZE+POLICY_SIZE+2))
samplescount =(csv_data.shape)[0]
cut_index = [(csv_data.shape)[1]-POLICY_SIZE-2, (csv_data.shape)[1]-2,(csv_data.shape)[1]-1]
samples,policy,value,countVisits=np.split(csv_data, cut_index,axis=1)
mask= np.where(policy < 0, -999999999.99, policy)
policy= np.where(policy < 0, 0, policy)

In [ ]:
np.set_printoptions(precision=5)
print(countVisits[0])
print(policy[0])
print(value[0])

In [ ]:
model.optimizer.learning_rate.assign(0.0005)

In [ ]:
print(samples.shape)
print(policy.shape)
print(value.shape)


In [ ]:
first20percent=int((samples.shape[0]*20/100))
first40percent=int((samples.shape[0]*50/100))
print(limit)
print(  samples[:first20percent].shape)
print(  samples[first20percent:first40percent].shape)
print(  samples[first40percent:].shape)
print(policy.shape)
print(value.shape)


In [ ]:
print("Training 20-40")
S40=samples[first20percent:first40percent]
P40=policy[first20percent:first40percent]
V40=value[first20percent:first40percent]     
model.optimizer.lr.assign(0.0005)
#model.fit({'input':S40}, {'policy': P40, 'value': V40},verbose=2, epochs=int(TRAINING_LOOPS*5/100),batch_size=256)        
print("Training 20")
firstS=samples[:first20percent]
firstP=policy[:first20percent]
firstV=value[:first20percent]    
if (generation < 3):
    model.optimizer.lr.assign(0.01)
else:        
    model.optimizer.lr.assign(0.001)
#model.fit({'input':firstS}, {'policy': firstP, 'value': firstV},verbose=2, epochs=int(TRAINING_LOOPS*10/100),batch_size=128)
model.optimizer.lr.assign(0.0001)
print("Running mini batches...",end='')
for r in range(int(TRAINING_LOOPS*85/100)):
    #minibatch
    selector=np.random.choice(value.shape[0], K_BATCH_SIZE, replace=False)
    miniS=samples[selector]
    miniP=policy[selector]
    miniV=value[selector]
    model.fit({'input':miniS}, {'policy': miniP, 'value': miniV},validation_data=({'input':firstS}, {'policy': firstP, 'value': firstV}),verbose=0, epochs=1)
print("done.")

In [ ]:
import shutil
shutil.copyfile(gen_best1+'.w32', 'A.w32')
shutil.copyfile(gen_best2+'.w32', 'B.w32')
#cpupt optimizer
BEST_CPUCT_MAX=1.00+random.random()*5.0    
BEST_CPUCT_MIN=(BEST_CPUCT_MAX-0.40)*random.random()+0.40
BEST_CPUCT_INC=0.001+0.035*random.random()
TEST_PARAM_THREAD= f"{THREADS} 150"
try:
    bestWinrate
except NameError:
    bestWinrate=0.0
with open('best_upct.txt', 'a') as f:
    f.write("Searching on best:"+gen_best1+"\n")
for upt in range(100):
    TEST_CPUCT_INC=0.001+0.035*random.random()
    TEST_CPUCT_MAX=1.00+random.random()*5.0    
    TEST_CPUCT_MIN=(TEST_CPUCT_MAX-1.00)*random.random()+1.00    
    if (random.random() < 0.5):
        TEST_CPUCT_INC=-TEST_CPUCT_INC
        T=TEST_CPUCT_MIN
        TEST_CPUCT_MIN=TEST_CPUCT_MAX
        TEST_CPUCT_MAX=T
    #pit
    #TEST_CPUCT_MAX=1.00+random.random()*5.0    
    #TEST_CPUCT_MIN=(TEST_CPUCT_MAX-1.00)*random.random()+1.00
    #TEST_CPUCT_INC=0.001+0.035*random.random()
    TEST_CPUCT_MIN=3.0-0.30+2.0*0.30*random.random()
    TEST_CPUCT_MAX=TEST_CPUCT_MIN+2.0*0.05*random.random()
    TEST_CPUCT_INC=0.00360-0.00080+2.0*0.00080*random.random()
    

    UPT_CC = f"{TEST_CPUCT_MIN} {TEST_CPUCT_INC} {TEST_CPUCT_MAX} {PIT_MCTS_ITER} {PIT_NOISE_DIR_EPSILON} {PIT_NOISE_DIR_ALPHA} {PIT_NOISE_DIR_DECAY} {PIT_NOISE_RANDOM} {PROPAGATE_BASE} {PROPAGATE_INC} {POLICY_BACKP_FIRST} {POLICY_BACKP_LAST}"    
    pitplay_process=os.path.join(".","CGZero")+" pitplay "+TEST_PARAM_THREAD+" A "+UPT_CC+" A "+PIT_PARAM_MCTS
    print('subprocess.run('+pitplay_process+', shell=True)')
    subprocess.run(pitplay_process, shell=True)
    pit_winrate=readAndDeleteWinrate("A","A")
    if (pit_winrate > 50.0 and pit_winrate > bestWinrate-5.0):
        pitplay_process=os.path.join(".","CGZero")+" pitplay "+TEST_PARAM_THREAD+" A "+UPT_CC+" B "+PIT_PARAM_MCTS
        subprocess.run(pitplay_process, shell=True)
        pit_winrate+=readAndDeleteWinrate("A","B")
        pit_winrate*=0.5
    with open('best_upct.txt', 'a') as f:
        f.write(f"{pit_winrate} | {TEST_CPUCT_MIN} {TEST_CPUCT_INC} {TEST_CPUCT_MAX} "+"\n")    
    print('Winrate '+str(pit_winrate))
    if (pit_winrate > bestWinrate):
        bestWinrate = pit_winrate
        BEST_CPUCT_MAX=TEST_CPUCT_MAX   
        BEST_CPUCT_MIN=TEST_CPUCT_MIN
        BEST_CPUCT_INC=TEST_CPUCT_INC
        print(f"New best {TEST_CPUCT_MIN} {TEST_CPUCT_INC} {TEST_CPUCT_MAX} {bestWinrate}% ")
        with open('best_upct.txt', 'a') as f:
            f.write(f"New best {bestWinrate}% | {TEST_CPUCT_MIN} {TEST_CPUCT_INC} {TEST_CPUCT_MAX}"+"\n") 

In [ ]:
csv_data = np.fromfile(SAMPLES_FILE, dtype=np.float32)
csv_data=np.reshape(csv_data, (-1,INPUT_SIZE+POLICY_SIZE+2))
cut_index = [(csv_data.shape)[1]-POLICY_SIZE-2, (csv_data.shape)[1]-2,(csv_data.shape)[1]-1]
samples,policy,value=np.split(csv_data, cut_index,axis=1)

mask= np.where(policy < 0, -999999999.99, policy)
policy= np.where(policy < 0, 0, policy) 

In [ ]:
print(policy)

In [ ]:
print(value)

In [ ]:
print(samples[-1])
bestWinrate=0

In [ ]:
model.optimizer.learning_rate.assign(0.0005)
model.fit({'input':samples}, {'policy': policy, 'value':value},verbose=2, epochs=int(60),callbacks=[csv_logger],batch_size=32)    

In [ ]:
#SaveModel(model,"superfit.w32")
VFF=1
predictions = model.predict(samples)
np.set_printoptions(suppress=True)
print(samples[VFF])
print("Value Predicted:",end='')
print("pred:"+str(predictions[0][VFF][0])+" real:"+str(value[VFF]))

print("Policy Predicted:",end='')
print(predictions[1][VFF])
print(policy[VFF])
print(np.abs(policy[VFF]-predictions[1][VFF]))

In [ ]:
print("y_pred=",end="")
print(predictions[1][8])
print(" y_true=",end="")
print(policy[8])
print(" diff=",end="")
print( np.abs(policy[8]-predictions[1][8]))

print(" categorical_crossentropy=",end="")
mse = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM)
mse(y_true, y_pred).numpy()
print(tf.keras.losses.categorical_crossentropy(policy[8],predictions[1][8] ).numpy())

print("y_pred=",end="")
print(predictions[1][8])
print(" y_true=",end="")
print(policy[8])
print(" KLD=",end="")
print(tf.keras.losses.KLD(policy[8],predictions[1][8]).numpy())

In [ ]:
ff=policy[8]- predictions[1][8]
print(ff*ff)
print(np.sum(ff*ff))

In [ ]:
tf.negative(tf.reduce_mean(tf.add(tf.multiply(policy[8],predictions[1][8]), 1)))

In [ ]:
tf.math.multiply(policy[8],predictions[1][8])

In [ ]:
y_pred=predictions[1][8]
y_true=policy[8]
print("y_pred=",end="")
print(y_pred)
print(" y_true=",end="")
print(y_true)
H=tf.nn.MeanSquaredError(predictions[1][8],policy[8],reduction="mean")
print(H.numpy())

In [ ]:
tf.reduce_mean(H)

In [ ]:
y_pred= np.array([0.0 ,0.0 , 0.0, 0.0,0.0, 1.0])
y_true= np.array([1.0, 0.0 , 0.0, 0.0,0.0,0.0])
print(tf.keras.losses.categorical_crossentropy(y_true,y_pred ).numpy())
print(tf.keras.losses.binary_crossentropy(y_true,y_pred ).numpy())
print(tf.keras.losses.KLD(y_true,y_pred ).numpy())
print("EEEEEEEEEEEEE")
y_pred= np.array([0.5 ,0.0 , 0.0, 0.0,0.0, 0.5])
y_true= np.array([1.0, 0.0 , 0.0, 0.0,0.0,0.0])
print(tf.keras.losses.categorical_crossentropy(y_true,y_pred ).numpy())
print(tf.keras.losses.binary_crossentropy(y_true,y_pred ).numpy())
print(tf.keras.losses.KLD(y_true,y_pred ).numpy())
print("EEEEEEEEEEEEE")
y_pred= np.array([0.45 ,0.45 , 0.0, 0.0,0.0, 0.10])
y_true= np.array([0.5, 0.5 , 0.0, 0.0,0.0,0.0])
print(tf.keras.losses.categorical_crossentropy(y_true,y_pred ).numpy())
print(tf.keras.losses.binary_crossentropy(y_true,y_pred ).numpy())
print(tf.keras.losses.KLD(y_true,y_pred ).numpy())


In [ ]:
mse = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
mse(y_true, y_pred).numpy()

In [ ]:
#subprocess.run('./NNSampler ./traindata3 Replay.*.dat ./traindata3/samples.dat 600000 400000 342 7 1', shell=True)
csv_data=None
samples=None
policy=None
value=None
csv_data = np.fromfile('./traindata3/d1.dat', dtype=np.float32)
csv_data=np.reshape(csv_data, (-1,INPUT_SIZE+POLICY_SIZE+2))
cut_index = [(csv_data.shape)[1]-POLICY_SIZE-2, (csv_data.shape)[1]-2,(csv_data.shape)[1]-1]
samples,policy,value=np.split(csv_data, cut_index,axis=1)
mask=np.where(policy < 0, -999999999.99, policy)
policy=np.where(policy < 0, 0, policy) 


In [ ]:
model.optimizer.learning_rate.assign(0.0001)
model.fit({'input':samples}, {'policy': policy, 'value':value},verbose=2, epochs=5000)    

In [ ]:
g1="gen"+str(1).zfill(4)
#model=tf.keras.models.load_model(g1+'.h5', custom_objects={"policy_loss": policy_loss})
#Saving
print('Save Model '+g1+'.h5')
model.save(g1+'.h5')
SaveModel(model,g1+".w32")

In [ ]:
salida=model.predict(samples)

In [ ]:
K=11
print(salida[1][K])
print(policy[K])
print(salida[0][K][0])
print(value[K][0])

In [ ]:
model.save(gen_name+'.h5')
SaveModel(model,gen_name+".w32")

In [ ]:
pitplay_process=os.path.join(".","CGZero")+" pitplay "+PIT_PARAM_THREAD+" "+gen_name+" "+PIT_PARAM_MCTS+" "+gen_best2+" "+PIT_PARAM_MCTS
print('subprocess.run('+pitplay_process+', shell=True)')
subprocess.run(pitplay_process, shell=True)

In [ ]:
pitplay_process=os.path.join(".","CGZero")+" pitplay "+PIT_PARAM_THREAD+" "+gen_name+" "+PIT_PARAM_MCTS+" "+gen_best2+" "+PIT_PARAM_MCTS
print('subprocess.run('+pitplay_process+', shell=True)')
subprocess.run(pitplay_process, shell=True)

In [ ]:
selfplay_process=os.path.join(".","CGZero")+" selfplay "+f"{THREADS} 500 "+gen_best1+" "+TRAIN_PARAMS+" "+gen_best2+" "+TRAIN_PARAMS
print(selfplay_process)
subprocess.run(selfplay_process, shell=True)

selfplay_process=os.path.join(".","CGZero")+" selfplay "+f"{THREADS} 500 "+gen_best2+" "+TRAIN_PARAMS+" "+gen_best2+" "+TRAIN_PARAMS
print(selfplay_process)
subprocess.run(selfplay_process, shell=True)
print("Done")

In [ ]:
csv_data = np.fromfile(SAMPLES_FILE, dtype=np.float32)
csv_data=np.reshape(csv_data, (-1,INPUT_SIZE+POLICY_SIZE+2))
cut_index = [(csv_data.shape)[1]-POLICY_SIZE-2, (csv_data.shape)[1]-2,(csv_data.shape)[1]-1]
samples,policy,value,countVisits=np.split(csv_data, cut_index,axis=1)

In [ ]:
print(countVisits)

In [ ]:
nn_1="gen0242"
nn_2="gen0239"
no_matches=300
custom=f"./CGZero selfplay {THREADS} {no_matches} {nn_1} 4.0 0.0034086042277349836 4.0 2000 0.24 1.3 0.0 0.05 0.9 0.1 20 5 {nn_2} 4.0 0.0034086042277349836 4.0 2000 0.24 1.3 0.0 0.05 0.9 0.1 20 5"
print(custom)
subprocess.run(custom, shell=True)